# Final Project Part B

In [65]:
# Install libraries
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from itertools import product
from IPython.display import display, HTML
from pandas_market_calendars import get_calendar 
import matplotlib.ticker as mtick # for the last few efficient frontier plots
import warnings
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
warnings.filterwarnings("ignore")

aspect_ratio = (12, 8)
annualization_factor=260

# 1

First let's define all the functions:

In [66]:
def read_price_data(filename,index_col=0, parse_dates=True):
    
    # Read the CSV file specifying that the first Date column is the index column 
    # Dates should be parsed as dateTimes    
    df = pd.read_csv(filename, index_col=index_col, parse_dates=parse_dates)
    
    # Print out the Dataframe’s info
    bold_text = '<strong>Dataframe info:</strong>'
    display(HTML(bold_text))
    print(df.info())
    
    # Check if any NULL values
    bold_text = '<strong>Null values check:</strong>'
    display(HTML(bold_text))
    print(df.isnull().sum())
    
    # Summary Statistics
    bold_text = '<strong>Summary Statistics:</strong>'
    display(HTML(bold_text))
    print(df.describe())
    
    # Display the Dataframe
    bold_text = '<strong>Display Dataframe:</strong>'
    display(HTML(bold_text))
    print(df)
    
    # Return the Dataframe
    return df

In [67]:
def plot_and_format_y_axis(data_frame, formatter = None, aspect_ratio = (12,8), title ='', ylabel = '', grid = True):
    
    ax = data_frame.plot(figsize=aspect_ratio, title=title, ylabel=ylabel, grid=grid)

    # Create a custom y-axis formatter
    if formatter != None:
        def currency_formatter(x, pos):
            if formatter == 'Dollar':
                return f"${x:,.2f}"  # Format as currency with 2 decimal places
            elif formatter == 'Percent':
                return f"{100*x:,.2f}%"  # Format as percentage with 2 decimal places

        yaxis_formatter = ticker.FuncFormatter(currency_formatter)
        ax.yaxis.set_major_formatter(yaxis_formatter)
    
    plt.show()

In [68]:
def print_pct_data(data_frame, format_spec = "{:.4%}", na_rep = "NaN", index_slice=None):
    
    # Create a copy of the input DataFrame
    formatted_df = data_frame.copy()

    if index_slice is not None:
        # Apply the formatting to the specified index_slice using applymap
        formatted_df.loc[index_slice] = formatted_df.loc[index_slice].applymap(lambda x: format_spec.format(x) if not pd.isna(x) else na_rep)
    
    else:
        formatted_df = data_frame.style.format(format_spec, na_rep = na_rep)

    return formatted_df

In [69]:
def insert_first_nan_row(returns_df):

    calendar_name = "XNYS"  # NYSE calendar
    start_date = pd.Timestamp(returns_df[0:1].index[0])
    num_weeks = 1  # 1 week before the start date

    # Get the specified calendar
    calendar = get_calendar(calendar_name)

    # Calculate the end date as 1 week before the start date
    end_date = start_date - pd.DateOffset(weeks=num_weeks)

    # Get the valid business days in the specified date range
    valid_business_days = calendar.valid_days(start_date=end_date, end_date=start_date)

    # Find the last but one date
    last_but_one_date = valid_business_days[-2]

    date_value = last_but_one_date.date()

    previous_bday_df = returns_df[0:1].copy()
    previous_bday_df.index = pd.DatetimeIndex([date_value])

    # Set the name of the index
    previous_bday_df.index.name = 'Date'

    # Replace all column values with NaN
    previous_bday_df[:] = np.nan

    # Concatenate the two DataFrames
    combined_df = pd.concat([previous_bday_df, returns_df])

    # Print the combined DataFrame
    return combined_df


In [70]:
def calcSummaryStatistics(returns_df, annualization_factor=260, print_screen=True, percent_format_string="{:.2%}"):
    
    # Check if the DataFrame has a first NaN row
    if not returns_df.iloc[0].isna().any():
        
        # Insert a first NaN row with a proper (t-1) business date
        returns_df = insert_first_nan_row(returns_df)
   
    if ( (returns_df.iloc[0].isna().any() and not returns_df.iloc[0].isna().all()) | (returns_df[1:].isna().any().any()) ):
        raise ValueError("DataFrame contains a first NaN row, but not all columns are NaN, OR it has a NaN row that is not the first row.")
        

    # Cumulate the log returns, replacing NaN with zeros
    cumReturns = returns_df.cumsum().fillna(0)
    underWaterCurve = cumReturns.fillna(0) - np.maximum.accumulate(cumReturns.fillna(0))


    # Calculate the Summary statistics
    summary_stats = pd.DataFrame([annualization_factor*returns_df.mean(), 
                                  np.sqrt(annualization_factor)*returns_df.std(),
                                  underWaterCurve.min(axis=0),
                                  (returns_df >= 0).sum() / (returns_df.dropna().shape[0]),
                                  returns_df.mean() / returns_df.std() * np.sqrt(annualization_factor)],
                                 index=['Average Annual Return', 'Annual Risk', 'maxDD', 'Success Ratio', 'Sharpe Ratio'])

    performance_statistics = print_pct_data(summary_stats, format_spec = percent_format_string, index_slice=pd.IndexSlice['Average Annual Return':'Success Ratio'])
        
    # Pretty print summary statistics
    if print_screen:
        print(performance_statistics)
    
    return performance_statistics

In [71]:
# Write function to implement moving average crossover strategy
# Input: Call this function with a dataframe having only one column representing prices of an instrument
# Output: Return a dataframe having three columns (original, go-flat, and go-short) of daily returns

def movingAverageCrossover(dataFrame, fastWindow, slowWindow):
   
    # Check to make sure the dataFrame has only 1 column 
    if len(dataFrame.columns) != 1:
        raise ValueError('The dataframe must only have 1 column.')
        
    # Name of the security
    security_name = dataFrame.columns[0]
    
    # Make a copy of the original dataframe
    df = dataFrame.copy()
    
    # Calculate fast and slow moving averages 
    df.loc[:,'FastSMA'] = df[security_name].rolling(window=fastWindow).mean()
    df.loc[:,'SlowSMA'] = df[security_name].rolling(window=slowWindow).mean()
    
    # Drop NaN values 
    df.dropna(inplace=True)
    
    
    ## Go flat strategy
    # SMA strategy: if rising trend, stay long, otherwise go flat
    df.loc[:,'Position_goflat'] = np.where(df['FastSMA'] > df['SlowSMA'], 1, 0)

    # Calculate daily returns 
    df.loc[:,'BMKReturns'] = np.log(df[security_name] / df[security_name].shift(1))
    
    # Then calculate the strategy returns, taking care to push down the Position column
    df.loc[:,'SMAReturns_goflat'] = df['Position_goflat'].shift(1) * df['BMKReturns']
    
    # Uncomment below to plot
    #ax = df[['BMKReturns', 'SMAReturns_goflat']].cumsum().plot(figsize=aspect_ratio, ylabel='NAV', title='Original and SMA Strategy Cumulative NAVs', grid=True)

    
    ## Go short strategy
    # SMA strategy: if rising trend, stay long, otherwise go short
    df.loc[:,'Position_goshort'] = np.where(df['FastSMA'] > df['SlowSMA'], 1, -1)

    # Then calculate the strategy returns, taking care to push down the Position column
    df.loc[:,'SMAReturns_goshort'] = df['Position_goshort'].shift(1) * df['BMKReturns']

    # Uncomment below to plot
    #ax = df[['BMKReturns', 'SMAReturns_goshort']].cumsum().plot(figsize=aspect_ratio, ylabel='NAV', title='Original and SMA Strategy Cumulative NAVs', grid=True)

    df = df[['BMKReturns', 'SMAReturns_goflat', 'SMAReturns_goshort']]

    # Rename the columns
    df.rename(columns={'BMKReturns': f"{security_name}-BMK",'SMAReturns_goflat': f"{security_name}-MAFlat",
       'SMAReturns_goshort': f"{security_name}-MAShort"}, inplace=True)

    return df


In [72]:
def bollingerBands(dataFrame, lookbackWindow, stdevBand):
    
    # Check to make sure the dataFrame has only 1 column 
    if len(dataFrame.columns) != 1:
        raise ValueError('The dataframe must only have 1 column.')
        
    # Name of the security
    security_name = dataFrame.columns[0]
    
    # Make a copy of the original dataframe
    df = dataFrame.copy()
    
    # Calculate the rolling mean and the rolling risk
    df.loc[:,'Mean'] = df[security_name].rolling(lookbackWindow).mean()
    df.loc[:,'Stdev'] = df[security_name].rolling(lookbackWindow).std()

    # Drop NaN values 
    df.dropna(inplace=True)
    
    # Next we create the Upper and Lower bands
    df.loc[:,'Upper'] = df['Mean'] + stdevBand * df['Stdev']
    df.loc[:,'Lower'] = df['Mean'] - stdevBand * df['Stdev']  

    # Now we implement the Bollinger band based strategy
    
    date0 = df.index[0]
    if df.loc[date0,security_name] >= df.loc[date0, 'Upper']:
        df.loc[date0, 'Position'] = -1
    elif df.loc[date0,security_name] <= df.loc[date0,'Lower']:
        df.loc[date0, 'Position'] = 1
    else:
        df.loc[date0, 'Position'] = 0

    # Then loop over the entire data set
    for i in range(1, df.shape[0]):
        today=df.index[i]
        yesterday=df.index[i-1]
        if df.loc[today,security_name] >= df.loc[today,'Upper']: # if close is above upper
            df.loc[today,'Position'] = -1 # then go short
        elif df.loc[today,security_name] <= df.loc[today,'Lower']: # if close is below lower
            df.loc[today,'Position'] = 1 # then go long
        elif df.loc[yesterday,'Position'] == -1 and df.loc[today,security_name] <= df.loc[today,'Mean']: # if prev day is short and we're now below the mean
            df.loc[today,'Position']=0 # then flatten
        elif df.loc[yesterday,'Position'] == 1 and df.loc[today, security_name] >= df.loc[today, 'Mean']: # conversely...
            df.loc[today,'Position']=0 # then also flatten
        else: # otherwise just hold yesterday's position
            df.loc[today,'Position']=df.loc[yesterday,'Position']
    
    
    # Now we calculate returns. First, the "benchmark" (or original) returns, the stock itself
    # first, calculate stock price log returns
    df.loc[:,'BMKReturns'] = np.log(df[security_name] / df[security_name].shift(1))
    
    # Then we calculate our strategy returns, taking care to multiply last night's position by today's benchmark return
    df.loc[:,'BBReturns'] = df['Position'].shift(1) * df['BMKReturns']
    
    df = df[['BMKReturns', 'BBReturns']]

    # Rename the columns
    df.rename(columns={'BMKReturns': f"{security_name}-BMK",'BBReturns': f"{security_name}-BB"}, inplace=True)

                       
    return df

Read in data from the file "PricesThruSep2023.csv":

In [73]:
prices_df = read_price_data('PricesThruSep2023.csv')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5975 entries, 1999-12-31 to 2023-09-29
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    5975 non-null   float64
 1   CAD     5975 non-null   float64
 2   EXC     5975 non-null   float64
 3   FBNDX   5975 non-null   float64
 4   GBP     5975 non-null   float64
 5   GE      5975 non-null   float64
 6   INTC    5975 non-null   float64
 7   PFE     5975 non-null   float64
 8   SPGSCI  5975 non-null   float64
 9   SPY     5975 non-null   float64
 10  VBTIX   5975 non-null   float64
 11  XPT     5975 non-null   float64
dtypes: float64(12)
memory usage: 606.8 KB
None


AAPL      0
CAD       0
EXC       0
FBNDX     0
GBP       0
GE        0
INTC      0
PFE       0
SPGSCI    0
SPY       0
VBTIX     0
XPT       0
dtype: int64


              AAPL          CAD          EXC        FBNDX          GBP  \
count  5975.000000  5975.000000  5975.000000  5975.000000  5975.000000   
mean     34.286590     0.820224    30.470551     7.574109     1.540253   
std      48.955902     0.113576    10.122516     0.464728     0.221106   
min       0.234286     0.619771    11.991797     6.170000     1.068900   
25%       2.106428     0.747133    23.380884     7.280000     1.352900   
50%      14.161786     0.790826    28.651926     7.570000     1.538500   
75%      39.405000     0.920132    35.638374     7.880000     1.650750   
max     196.449997     1.086484    65.363770     8.870000     2.107500   

                GE         INTC          PFE       SPGSCI          SPY  \
count  5975.000000  5975.000000  5975.000000  5975.000000  5975.000000   
mean    149.622005    32.022611    30.256815   448.968465   194.683556   
std      68.259282    13.160513     9.121915   157.161629   103.117757   
min      34.285713    12.080000    11

                  AAPL       CAD        EXC  FBNDX     GBP          GE  \
Date                                                                     
1999-12-31    0.917969  0.691515  12.393010   6.89  1.6182  309.753998   
2000-01-03    0.999442  0.691181  12.080956   6.85  1.6362  300.246216   
2000-01-04    0.915179  0.688326  11.991797   6.87  1.6360  288.236359   
2000-01-05    0.928571  0.689655  12.571327   6.84  1.6426  287.735931   
2000-01-06    0.848214  0.684978  12.593616   6.86  1.6474  291.582855   
...                ...       ...        ...    ...     ...         ...   
2023-09-25  176.080002  0.743273  40.299999   6.87  1.2211  111.720001   
2023-09-26  171.960007  0.739809  39.150002   6.86  1.2158  109.930000   
2023-09-27  170.429993  0.740850  38.799999   6.84  1.2135  110.889999   
2023-09-28  170.690002  0.741455  37.880001   6.85  1.2203  112.339996   
2023-09-29  171.210007  0.736540  37.790001   6.85  1.2199  110.550003   

                 INTC        PFE    S

# 2.

To have an idea on initial Sharpe values, display performance statistics:

In [74]:
# Compute log returns
logReturns = np.log(prices_df / prices_df.shift(1))
prices_stats = calcSummaryStatistics(logReturns, print_screen=True)

                           AAPL       CAD       EXC     FBNDX       GBP  \
Average Annual Return    22.76%     0.27%     4.85%    -0.03%    -1.23%   
Annual Risk              41.52%     8.87%    26.74%     4.25%     9.57%   
maxDD                  -170.38%   -45.99%  -128.08%   -26.43%   -67.89%   
Success Ratio            52.54%    50.62%    52.71%    63.61%    50.49%   
Sharpe Ratio           0.548073  0.030956  0.181468 -0.005957 -0.128515   

                             GE      INTC       PFE    SPGSCI       SPY  \
Average Annual Return    -4.48%    -0.64%     0.33%     4.97%     4.65%   
Annual Risk              33.92%    38.32%    25.71%    24.07%    20.05%   
maxDD                  -235.22%  -182.43%  -143.57%  -136.11%   -83.18%   
Success Ratio            50.20%    51.47%    50.12%    52.68%    54.03%   
Sharpe Ratio          -0.132183 -0.016633  0.012685  0.206524  0.231886   

                          VBTIX       XPT  
Average Annual Return    -0.18%     3.13%  
Annual Ris

### Choices Selected for the securities in Part A:

- Any 5 equity instruments: AAPL-MAFLAT, GE-MAShort, INTC-BB, PFE-BB, SPY-MAFlat
- 1 of the 2 fixed income instruments: FBN-MAShort 
- 1 of the 2 commodity instruments: XPT-MAShort
- 1 of the 2 currency pairs: GBP-MAShort


Re-running Part A chosen portfolio (chosen instruments with the chosen technical strategies with their chosen lookbacks):

In [75]:
# Equities:

# Define the parameters
fastWindow = 11
slowWindow = 220
lookback_window = 22
stdev_band = 2

# Apply MA crossover and Bollinger Band strategies
apple_df = movingAverageCrossover(pd.DataFrame(prices_df['AAPL']), fastWindow, slowWindow)
apple_df_stats = calcSummaryStatistics(apple_df, print_screen=True)

ge_df = movingAverageCrossover(pd.DataFrame(prices_df['GE']), fastWindow, slowWindow)
ge_df_stats = calcSummaryStatistics(ge_df, print_screen=True)

spy_df = movingAverageCrossover(pd.DataFrame(prices_df['SPY']), fastWindow, slowWindow)
spy_df_stats = calcSummaryStatistics(spy_df, print_screen=True)

intc_bb_df = bollingerBands(pd.DataFrame(prices_df['INTC']), lookback_window, stdev_band)
intc_bb_df_stats = calcSummaryStatistics(intc_bb_df, print_screen=True)

pfe_bb_df = bollingerBands(pd.DataFrame(prices_df['PFE']), lookback_window, stdev_band)
pfe_bb_df_stats = calcSummaryStatistics(pfe_bb_df, print_screen=True)


                       AAPL-BMK AAPL-MAFlat AAPL-MAShort
Average Annual Return    28.10%      26.73%       25.36%
Annual Risk              36.72%      27.76%       36.73%
maxDD                   -93.82%     -55.13%      -76.07%
Success Ratio            52.63%      65.32%       52.84%
Sharpe Ratio           0.765383     0.96304     0.690537
                         GE-BMK GE-MAFlat GE-MAShort
Average Annual Return    -4.85%     1.78%      8.42%
Annual Risk              33.77%    16.86%     33.77%
maxDD                  -227.09%   -80.59%   -139.65%
Success Ratio            50.20%    74.60%     51.36%
Sharpe Ratio          -0.143633  0.105725   0.249219
                        SPY-BMK SPY-MAFlat SPY-MAShort
Average Annual Return     5.15%      5.83%       6.50%
Annual Risk              19.89%     11.37%      19.89%
maxDD                   -83.18%    -24.05%     -50.83%
Success Ratio            54.23%     69.12%      53.36%
Sharpe Ratio           0.259027   0.512578    0.326772
          

In [76]:
# Other assets

# Define the parameters
fastWindow = 93
slowWindow = 104

# Apply MA crossover strategy
fbndx_df = movingAverageCrossover(pd.DataFrame(prices_df['FBNDX']), fastWindow, slowWindow)
fbndx_df_stats = calcSummaryStatistics(fbndx_df, print_screen=True)

xpt_df = movingAverageCrossover(pd.DataFrame(prices_df['XPT']), fastWindow, slowWindow)
xpt_df_stats = calcSummaryStatistics(xpt_df, print_screen=True)

gbp_df = movingAverageCrossover(pd.DataFrame(prices_df['GBP']), fastWindow, slowWindow)
gbp_df_stats = calcSummaryStatistics(gbp_df, print_screen=True)

                      FBNDX-BMK FBNDX-MAFlat FBNDX-MAShort
Average Annual Return     0.04%        1.18%         2.31%
Annual Risk               4.26%        2.98%         4.25%
maxDD                   -26.43%       -9.89%       -12.36%
Success Ratio            63.72%       81.28%        63.77%
Sharpe Ratio           0.009156     0.395428      0.544212
                        XPT-BMK XPT-MAFlat XPT-MAShort
Average Annual Return     2.18%      5.53%       8.87%
Annual Risk              23.87%     17.22%      23.87%
maxDD                  -133.68%   -105.95%    -155.81%
Success Ratio            52.36%     73.28%      52.17%
Sharpe Ratio           0.091295   0.320918     0.37175
                        GBP-BMK GBP-MAFlat GBP-MAShort
Average Annual Return    -0.91%      0.85%       2.61%
Annual Risk               9.59%      5.98%       9.59%
maxDD                   -67.89%    -21.39%     -31.53%
Success Ratio            50.55%     75.23%      50.98%
Sharpe Ratio          -0.094431   0.14285

Creating a new dataFrame which has the daily returns of the 8 chosen strategies with below format:
- columns named appropriately.
- columns ordered as follows: first, the chosen 5 equities in alphabetical order, then fixed income, then commodity, and finally currency.

In [77]:
# Perform the inner join of the necessary columns
portfolio_returns_df = pd.DataFrame(apple_df['AAPL-MAFlat']).join([
    ge_df['GE-MAShort'],
    intc_bb_df['INTC-BB'],
    pfe_bb_df['PFE-BB'],
    spy_df['SPY-MAFlat'],
    fbndx_df['FBNDX-MAShort'],
    xpt_df['XPT-MAShort'],
    gbp_df['GBP-MAShort']
], how='inner')

# Note that the first row has Nan values, dropping the row to calculate stats and compare
portfolio_returns_df = portfolio_returns_df.dropna() 

# Calculate performance statistics for the combined DataFrame
portfolio_returns_df_stats = calcSummaryStatistics(portfolio_returns_df, print_screen=False)
portfolio_returns_df_stats

,AAPL-MAFlat,GE-MAShort,INTC-BB,PFE-BB,SPY-MAFlat,FBNDX-MAShort,XPT-MAShort,GBP-MAShort
Average Annual Return,26.73%,8.42%,11.50%,4.30%,5.83%,2.32%,8.65%,2.45%
Annual Risk,27.76%,33.77%,27.30%,19.36%,11.37%,4.27%,23.85%,9.58%
maxDD,-55.13%,-139.65%,-56.87%,-80.32%,-24.05%,-12.36%,-155.81%,-31.53%
Success Ratio,65.32%,51.36%,73.99%,73.52%,69.12%,63.70%,52.13%,50.95%
Sharpe Ratio,0.96304,0.249219,0.421362,0.222219,0.512578,0.543289,0.362527,0.255287


##### List out the column names:

In [78]:
myPortfolioColumns = list(portfolio_returns_df.columns)
myPortfolioColumns

['AAPL-MAFlat',
 'GE-MAShort',
 'INTC-BB',
 'PFE-BB',
 'SPY-MAFlat',
 'FBNDX-MAShort',
 'XPT-MAShort',
 'GBP-MAShort']

Verifying that the start date is same as in Part A (2000-11-13):

In [79]:
portfolio_returns_df

,AAPL-MAFlat,GE-MAShort,INTC-BB,PFE-BB,SPY-MAFlat,FBNDX-MAShort,XPT-MAShort,GBP-MAShort
Date,,,,,,,,
2000-11-13,0.000000,-0.046299,0.0,-0.000000,-0.000000,0.001430,-0.009368,-0.008731
2000-11-14,0.000000,0.025196,0.0,0.000000,0.000000,0.001428,0.008368,0.005020
2000-11-15,-0.000000,-0.004751,0.0,0.000000,0.000000,0.001426,-0.008368,0.002660
2000-11-16,-0.000000,0.002378,-0.0,0.000000,-0.000000,0.001423,-0.014048,0.000982
2000-11-17,-0.000000,-0.014354,0.0,-0.000000,-0.000000,-0.001423,0.009837,0.001966
...,...,...,...,...,...,...,...,...
2023-09-25,0.007353,0.004216,0.0,0.008832,0.004196,0.007252,0.017330,-0.002454
2023-09-26,-0.023677,-0.016152,-0.0,-0.017743,-0.014800,0.001457,0.007792,-0.004350
2023-09-27,-0.008937,0.008695,0.0,-0.009303,0.000399,0.002920,0.017191,-0.001894


# 3.

MVO maximum-Sharpe weights that were obtained in Part A:

In [80]:
max_sharpe_weights = np.array([0.17143388, 0.0762359 , 0.06332   , 0.17258526, 0.03777947, 0.16811505, 0.13946747, 0.17106296])

df = pd.DataFrame(max_sharpe_weights)
df.columns = ['Weights']
df['Security'] = myPortfolioColumns
df.set_index('Security', inplace=True)
print_pct_data(df)

,Weights
Security,
AAPL-MAFlat,17.1434%
GE-MAShort,7.6236%
INTC-BB,6.3320%
PFE-BB,17.2585%
SPY-MAFlat,3.7779%
FBNDX-MAShort,16.8115%
XPT-MAShort,13.9467%
GBP-MAShort,17.1063%


Using the same MVO maximum-Sharpe weights, adding a 9th column to the dataFrame which is the MVO-weighted maximum-Sharpe portfolio:

In [81]:
max_sharpe_portfolio = pd.DataFrame((max_sharpe_weights * portfolio_returns_df).sum(axis=1, skipna=False), columns=['Max Sharpe Portfolio'])
max_sharpe_portfolio = portfolio_returns_df.join(max_sharpe_portfolio, how='inner')

Calculate the performance statistics for two sub-periods:

First sub-period is identical to the Part A until 12-31-2018:

In [82]:
df = max_sharpe_portfolio.copy()

# Split the DataFrame based on the date '2019-01-02'
date_to_split = '2019-01-02'
part_a_df = df.loc[df.index < date_to_split]
out_of_sample_df = df.loc[df.index >= date_to_split]

Verify that the first sub-period Sharpe is identical to what you got in Part A:

In [83]:
max_sharpe_stats1 = calcSummaryStatistics(part_a_df, print_screen=False)
max_sharpe_stats1

,AAPL-MAFlat,GE-MAShort,INTC-BB,PFE-BB,SPY-MAFlat,FBNDX-MAShort,XPT-MAShort,GBP-MAShort,Max Sharpe Portfolio
Average Annual Return,27.81%,12.14%,10.82%,9.06%,5.46%,1.47%,14.30%,4.30%,11.12%
Annual Risk,27.47%,31.02%,25.95%,18.56%,10.58%,3.80%,22.16%,9.52%,7.59%
maxDD,-55.13%,-90.44%,-45.33%,-30.51%,-19.50%,-12.36%,-66.89%,-31.53%,-8.80%
Success Ratio,65.92%,51.80%,74.71%,74.19%,69.56%,63.84%,52.63%,51.32%,53.97%
Sharpe Ratio,1.012515,0.391275,0.416923,0.487879,0.515852,0.387584,0.645098,0.451441,1.466141


It is exactly same as the max sharpe portfolio that we have got in Part A.

Second sub-period is the out-of-sample period 2019-01-02 through 2023-09-29:

In [84]:
max_sharpe_stats2 = calcSummaryStatistics(out_of_sample_df, print_screen=False)
max_sharpe_stats2

,AAPL-MAFlat,GE-MAShort,INTC-BB,PFE-BB,SPY-MAFlat,FBNDX-MAShort,XPT-MAShort,GBP-MAShort,Max Sharpe Portfolio
Average Annual Return,22.61%,-5.78%,14.11%,-13.84%,7.23%,5.56%,-12.91%,-4.62%,0.56%
Annual Risk,28.85%,42.67%,31.94%,22.11%,13.97%,5.72%,29.39%,9.81%,9.97%
maxDD,-37.73%,-127.59%,-56.87%,-80.32%,-24.05%,-9.03%,-106.62%,-24.96%,-21.83%
Success Ratio,63.01%,49.71%,71.21%,70.96%,67.45%,63.18%,50.21%,49.54%,51.21%
Sharpe Ratio,0.783829,-0.135551,0.441817,-0.626079,0.51767,0.970908,-0.439346,-0.470318,0.05591


Sharpe for the new active portfolio's out-of-sample period is <b>0.05591</b>. It is less than the Part A first sub-period sharpe.

# 4.

Constructing an equal-weighted portfolio of the benchmark versions of the chosen instruments:

In [85]:
# Perform the inner join of the necessary columns
portfolio_returns_BMK = pd.DataFrame(logReturns['AAPL']).join([
    logReturns['GE'],
    logReturns['INTC'],
    logReturns['PFE'],
    logReturns['SPY'],
    logReturns['FBNDX'],
    logReturns['XPT'],
    logReturns['GBP']
], how='inner')

portfolio_returns_BMK = portfolio_returns_BMK.loc[portfolio_returns_BMK.index > '2018-12-31']
portfolio_returns_BMK

,AAPL,GE,INTC,PFE,SPY,FBNDX,XPT,GBP
Date,,,,,,,,
2019-01-02,0.001140,0.061479,0.003191,-0.009206,0.001040,0.001303,-0.000365,-0.011593
2019-01-03,-0.104924,0.001242,-0.056584,-0.028376,-0.024152,0.005195,0.003000,0.001664
2019-01-04,0.041803,0.020872,0.059553,0.022579,0.032947,-0.003894,0.030796,0.007495
2019-01-07,-0.002228,0.060124,0.004648,0.005335,0.007854,-0.001301,-0.001241,0.004235
2019-01-08,0.018884,-0.020810,0.006304,0.004616,0.009351,0.000000,-0.002779,-0.004707
...,...,...,...,...,...,...,...,...
2023-09-25,0.007353,0.004216,0.003213,0.008832,0.004196,-0.007252,-0.017330,-0.002454
2023-09-26,-0.023677,-0.016152,-0.013506,-0.017743,-0.014800,-0.001457,-0.007792,-0.004350
2023-09-27,-0.008937,0.008695,0.022795,-0.009303,0.000399,-0.002920,-0.017191,-0.001894


In [86]:
# Creating an equal weight vector
equal_weights = np.array([1 / portfolio_returns_BMK.shape[1]] * portfolio_returns_BMK.shape[1])


bmk_portfolio = pd.DataFrame((equal_weights * portfolio_returns_BMK).sum(axis=1, skipna=False), columns=['Equal-weighted Portfolio'])
bmk_portfolio = portfolio_returns_BMK.join(bmk_portfolio, how='inner')
bmk_portfolio

,AAPL,GE,INTC,PFE,SPY,FBNDX,XPT,GBP,Equal-weighted Portfolio
Date,,,,,,,,,
2019-01-02,0.001140,0.061479,0.003191,-0.009206,0.001040,0.001303,-0.000365,-0.011593,0.005874
2019-01-03,-0.104924,0.001242,-0.056584,-0.028376,-0.024152,0.005195,0.003000,0.001664,-0.025367
2019-01-04,0.041803,0.020872,0.059553,0.022579,0.032947,-0.003894,0.030796,0.007495,0.026519
2019-01-07,-0.002228,0.060124,0.004648,0.005335,0.007854,-0.001301,-0.001241,0.004235,0.009678
2019-01-08,0.018884,-0.020810,0.006304,0.004616,0.009351,0.000000,-0.002779,-0.004707,0.001357
...,...,...,...,...,...,...,...,...,...
2023-09-25,0.007353,0.004216,0.003213,0.008832,0.004196,-0.007252,-0.017330,-0.002454,0.000097
2023-09-26,-0.023677,-0.016152,-0.013506,-0.017743,-0.014800,-0.001457,-0.007792,-0.004350,-0.012434
2023-09-27,-0.008937,0.008695,0.022795,-0.009303,0.000399,-0.002920,-0.017191,-0.001894,-0.001044


Calculate summary statistics of the "passive" portfolio:

In [87]:
bmk_portfolio_stats = calcSummaryStatistics(bmk_portfolio, print_screen=False)
bmk_portfolio_stats

,AAPL,GE,INTC,PFE,SPY,FBNDX,XPT,GBP,Equal-weighted Portfolio
Average Annual Return,31.94%,19.34%,-6.04%,-4.83%,11.68%,-2.46%,2.87%,-0.97%,6.44%
Annual Risk,33.41%,42.66%,39.72%,27.11%,21.86%,5.73%,29.40%,9.82%,17.74%
maxDD,-37.73%,-87.43%,-101.15%,-64.64%,-41.71%,-26.43%,-54.80%,-28.45%,-33.80%
Success Ratio,53.72%,51.05%,50.46%,48.70%,54.56%,59.16%,50.88%,49.62%,53.56%
Sharpe Ratio,0.956061,0.453289,-0.152109,-0.178135,0.534244,-0.429167,0.097659,-0.098605,0.363194


Passive benchmark-only Sharpe ratio for the new out-of-sample period is <b>0.3632</b>.

# 5.

a) The active portfolio's performance for the out-of-sample period in terms of sharpe(0.055) is less compared to that of Part A.

b) Active portfolio's out-of-sample Sharpe ratio is 0.055 and it does not outperform the equally-weighted benchmark's Sharpe ratio of 0.3632. It suggests that the active management strategy did not add value over the period analyzed. This does not render the exercise futile; rather, it offers valuable insights. It highlights the need for strategy reevaluation, overfitting analysis, and potentially adjusting the risk-return profile. Learning from underperformance is as crucial as profiting from successful predictions, providing a foundation for improving future strategies.

A significant change in the Sharpe ratio from one period to another, can be due to several factors:
- Weight Allocations: The weights for each of the instruments in portfolio were optimized based on the dataset being used to maximize Sharpe. With out-of-sample dataset, the weight allocation doesn't seem to perform that well indicating that mean-variance optimization might require a rerun for the updated dataset.
- Window-Selection results: Fast window, Slow window and Lookback window were all chosen to optimize Sharpe in Part A. These windows might not result in good Sharpe ratios for updated dataset.
- Market Conditions: Different market conditions between the two periods can impact the performance of trading strategies. The period from 2000 to 2018 saw various market phases, while 2019 to 2023 may have had different volatility, trends, or economic conditions affecting the securities' performance.
- The underlying data distributions and relationships between securities may have shifted due to changes in the economy, industry dynamics, or other systemic factors.
- It may also reflect that the strategy is not robust enough to adapt to new market conditions or that the risk-return profile of the assets has changed.

It's not uncommon for strategies to perform differently as time progresses due to these reasons. Continuous strategy evaluation and adaptation to current market conditions are essential for maintaining a good Sharpe ratio.

"We pledge on our honor that we have not given nor received any unauthorized assistance on this
assignment to/from other groups. Nor have we received any assistance from prior year students."
- Lakshita Garg, Satvik Narang, Thanmai Reddy, Pruthvi Billa, Kristina Sutliff